In [1]:
import numpy as np
import pandas as pd
import cv2
from skimage import feature
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import os

# Load the dataset

In [2]:
train_data_path = 'archive/train'  # Path to the training data folder
test_data_path = 'archive/test'    # Path to the testing data folder
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
X_train = []
y_train = []
X_test = []
y_test = []

# Load training data
for emotion_label, emotion in enumerate(emotions):
    emotion_folder = os.path.join(train_data_path, emotion)
    for image_filename in os.listdir(emotion_folder):
        image_path = os.path.join(emotion_folder, image_filename)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (48, 48))

        # Divide the image into 4x4 blocks
        block_size = 12
        for i in range(4):
            for j in range(4):
                block = img[i * block_size:(i + 1) * block_size, j * block_size:(j + 1) * block_size]
                lbp_features = feature.local_binary_pattern(block, P=8, R=1, method='uniform')
                lbp_hist, _ = np.histogram(lbp_features, bins=np.arange(0, 60), density=True)
                X_train.append(lbp_hist)  # Add the LBP histogram to X_train
                y_train.append(emotion_label)  # Add the emotion label to y_train

# Load testing data
for emotion_label, emotion in enumerate(emotions):
    emotion_folder = os.path.join(test_data_path, emotion)
    for image_filename in os.listdir(emotion_folder):
        image_path = os.path.join(emotion_folder, image_filename)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (48, 48))

        # Divide the image into 4x4 blocks and compute LBP histograms
        for i in range(4):
            for j in range(4):
                block = img[i * block_size:(i + 1) * block_size, j * block_size:(j + 1) * block_size]
                lbp_features = feature.local_binary_pattern(block, P=8, R=1, method='uniform')
                lbp_hist, _ = np.histogram(lbp_features, bins=np.arange(0, 60), density=True)
                X_test.append(lbp_hist)  # Add the LBP histogram to X_test
                y_test.append(emotion_label)  # Add the emotion label to y_test

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# KNN classifier

In [3]:
n_neighbors = 5
knn_classifier = KNeighborsClassifier(n_neighbors=n_neighbors)
knn_classifier.fit(X_train, y_train)

KNeighborsClassifier()

# Calculate accuracy

In [4]:
y_pred = knn_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 20.39%


In [5]:
crosstab = pd.crosstab(y_test, y_pred, rownames=['Actual types'], colnames=['Predicted Types'])
print(crosstab) 

Predicted Types     0    1     2     3     4     5     6
Actual types                                            
0                3760  185  2255  4631  2004  1629   864
1                 344  222   268   455   192   189   106
2                3315  188  3095  4951  2158  1669  1008
3                5916  345  4363  9288  3762  3014  1696
4                4093  213  2968  6184  2943  2173  1154
5                4055  224  3076  6305  2763  2392  1137
6                2558  140  1981  3836  1671  1390  1720
